In [1]:
class thermal_generator:
    
    def __init__(self,power_out_min, power_out_max, time_up_min, time_down_min,must_run, 
                 ramp_up, ramp_down, cost, A_D, A_U, time_horizon ):
        self.time_up_min = time_up_min
        self.time_down_min = time_down_min
        self.must_run = must_run
        self.ramp_up = ramp_up
        self.ramp_down = ramp_down
        self.cost = cost
        self.A_D = A_D
        self.A_U = A_U
        self.power_level = []
        for level in range(0,time_horizon):
            self.power_level.append([0] + [x for x in range(round(power_out_min), round(power_out_max + 1))])
    
    def print_all_info(self):
        print("Minimum time up : {}".format(self.time_up_min))
        print("Minimum time down : {}".format(self.time_down_min))
        print("Must run time : {}".format(self.must_run))
        print("Ramp up : {}".format(self.ramp_up))
        print("Ramp down : {}".format(self.ramp_down))
        print("Costs : {} ".format(self.cost))
        print("Start up cost : {}".format(self.A_U))
        print("Start down cost : {}".format(self.A_D))
        print("Power levels : {}".format(self.power_level))

In [2]:
class wind_generator:
    
    def __init__(self,power_levels):
        self.power_level = []
        for level in power_levels:
            self.power_level.append([0] + [x for x in range(round(level[0]), round(level[1] + 1))])
        self.time_up_min = 0
        self.time_down_min = 0
        self.must_run = 0
        self.ramp_up = 0
        self.ramp_down = 0
        self.cost = 0
        self.A_D = 0
        self.A_U = 0
        
    def print_all_info(self):
        print("power maximum output : {}".format(self.power_level))

In [3]:
class network:
    
    def __init__(self, time_period, demand, reserves, thermal_gen = [], renewable_gen = []):
        self.time_period = time_period
        self.demand = demand
        self.reserves = reserves
        self.thermal_gen = thermal_gen
        self.renewable_gen = renewable_gen
        self.gen = []
    
    def add_thermal_generator(self, thermal_generator):
        self.thermal_gen.append(thermal_generator)
    
    def add_renewable_generator(self, renew_generator):
        self.renewable_gen.append(renew_generator)
    
    def get_all_generators(self):
        self.gen = self.renewable_gen + self.thermal_gen
    
    def print_all_info(self):
        print("Time period : {}".format(self.time_period))
        print("Demand(Loads) over time : {}".format(self.demand))
        print("reserves over time : {}".format(self.reserves))
        print("Number of Thermal generators : {}".format(len(self.thermal_gen)))
        print("Number of renewable generators : {}".format(len(self.renewable_gen)))

In [4]:
def create_thermal_generator(json_object,time):
    return thermal_generator(json_object['power_output_minimum'],json_object['power_output_maximum'],
                             json_object['time_up_minimum'], json_object['time_down_minimum'],
                            json_object['must_run'],json_object['ramp_up_limit'],json_object['ramp_down_limit'],
                            json_object['piecewise_production'][0]['cost'],
                            json_object['startup'][0]['cost'],
                            0,time)

def create_renewable_generator(json_object):
    power_levels = []
    for x in range(0,len(json_object['power_output_minimum'])):
        power_levels.append((json_object['power_output_minimum'][x],json_object['power_output_maximum'][x]))
    return wind_generator(power_levels)

def create_network(json_object, time):
    net = network(json_object['time_periods'],json_object['demand'],json_object['reserves'])
    for x in json_object['thermal_generators']:
        gen = create_thermal_generator(json_object['thermal_generators'][x],time)
        net.add_thermal_generator(gen)
    for x in json_object['renewable_generators']:
        gen = create_renewable_generator(json_object['renewable_generators'][x])
        net.add_renewable_generator(gen)
    return net

In [5]:
alphabet = {0:"a",1:"b",2:"c",3:"d",4:"e",5:"f",6:"g",7:"h",8:"i",9:"j"}

def map_variable(i,k,t, K, T):
    return (i* T*K) + (t*K) + k

def return_var_name(i,k,t):
    var_name = "1{}{}{}".format(i,k,t)
    return int(var_name)

def create_linear_dict(net):
    dic ={}
    for t in range(0, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                dic[return_var_name(i,k,t)] = 0
                #print("adding{}{}{}".format(i,k,t))
    return dic

def create_quadratic_dict(net):
    dic ={}
    for t in range(0, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                for z in range(0, net.time_period):
                    for j in range(0, len(net.gen)):
                        for l in range(0, len(net.gen[i].power_level[t])):
                            dic[(return_var_name(i,k,t),return_var_name(j,l,z))] = 0
    return dic

In [6]:
import json
#from model import *
# JSON file
f = open ('<jsonfile.path>', "r")
# Reading from file
data = json.loads(f.read())
net = create_network(data, time = 16)
net.get_all_generators()
net.print_all_info()
linear_terms = create_linear_dict(net)
quadratic_terms = create_quadratic_dict(net)

Time period : 12
Demand(Loads) over time : [26089.9326691, 24964.3953213, 24325.3179734, 24163.1706255, 24719.2732776, 25997.3159298, 27963.1685819, 29824.691234, 31948.2538862, 34162.5465383, 36242.5691904, 38159.9918426]
reserves over time : [260.899326691, 249.643953213, 243.253179734, 241.63170625499998, 247.192732776, 259.973159298, 279.63168581900004, 298.24691234000005, 319.482538862, 341.625465383, 362.42569190399996, 381.599918426]
Number of Thermal generators : 3
Number of renewable generators : 3


In [7]:
constant = 0
for t in range(0, net.time_period):
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            linear_terms[return_var_name(i,k,t)] = net.gen[i].cost
            if t == 0:
                linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)] + net.gen[i].A_U
            elif t > 0:
                linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)] + net.gen[i].A_U
                #print(i,k,t)
                linear_terms[return_var_name(i,min(k,len(net.gen[i].power_level[t-1])-1),t-1)] = linear_terms[return_var_name(i,min(k,len(net.gen[i].power_level[t-1])-1),t-1)] + net.gen[i].A_D
                quadratic_terms[(return_var_name(i,k,t),return_var_name(i,min(k,len(net.gen[i].power_level[t-1])-1),t-1))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,min(k,len(net.gen[i].power_level[t-1])-1),t-1))] + (-1* net.gen[i].A_U* net.gen[i].A_D)
for t in range(0, net.time_period):
    constant = constant + int(net.demand[t] * net.demand[t])
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)] + int(-2 * net.demand[t] *net.gen[i].power_level[t][k])
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] + (net.gen[i].power_level[t][k]*net.gen[i].power_level[t][k])
            for j in range(0, i):
                for z in range(0, min(len(net.gen[j].power_level[t]),k)):
                    quadratic_terms[(return_var_name(i,k,t),return_var_name(j,z,t))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(j,z,t))] + (2*net.gen[i].power_level[t][k]*net.gen[j].power_level[t][z])
for t in range(0, net.time_period):
    constant = constant + 1
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)]-2
            quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] + 1
for t in range(1, net.time_period):
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            constant = constant + (1 + net.gen[i].time_up_min) * (1 + net.gen[i].time_up_min)
            for t_mut in range(t, min(t + net.gen[i].time_up_min, net.time_period)):
                linear_terms[return_var_name(i,k,t_mut)] = linear_terms[return_var_name(i,k,t_mut)] + ((-2) * (1 +net.gen[i].time_up_min))
                for j in range(0, i):
                    for z in range(0, min(len(net.gen[j].power_level[t]),k)):
                        quadratic_terms[(return_var_name(i,k,t_mut),return_var_name(j,z,t_mut))] = quadratic_terms[(return_var_name(i,k,t_mut),return_var_name(j,z,t_mut))] + (2*net.gen[i].power_level[t][k]*net.gen[j].power_level[t][z])
for t in range(1, net.time_period):
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            constant = constant + 1 
            for t_mut in range(t, min(t + net.gen[i].time_down_min, net.time_period)):
                linear_terms[return_var_name(i,k,t_mut)] = linear_terms[return_var_name(i,k,t_mut)] -2
                for j in range(0, i):
                    for z in range(0, min(len(net.gen[j].power_level[t]),k)): 
                        quadratic_terms[(return_var_name(i,k,t_mut),return_var_name(j,z,t_mut))] = quadratic_terms[(return_var_name(i,k,t_mut),return_var_name(j,z,t_mut))] + (2*net.gen[i].power_level[t][k]*net.gen[j].power_level[t][z])
for t in range(1, net.time_period):
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            for l in range(0, len(net.gen[i].power_level[t])):
                constant = constant  + (int(1 - net.gen[i].ramp_up) * int(1 - net.gen[i].ramp_up))
                linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)] + (2 * (1 -net.gen[i].ramp_up ) * net.gen[i].power_level[t][k])
                linear_terms[return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1)] = linear_terms[return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1)] + (2 * (1 -net.gen[i].ramp_up ) * -net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)])
                quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] + (net.gen[i].power_level[t][k]* net.gen[i].power_level[t][k])
                quadratic_terms[(return_var_name(i,l,t-1),return_var_name(i,l,t-1))] = quadratic_terms[(return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1),return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1))] + (net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)]* net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)])
                quadratic_terms[(return_var_name(i,k,t),return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1))] - 2*(net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)]* net.gen[i].power_level[t][k])
for t in range(1, net.time_period):
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):
            for l in range(0, len(net.gen[i].power_level[t])):
                constant = constant  + (int(1 - net.gen[i].ramp_down) * int(1 - net.gen[i].ramp_up))
                linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)] + (2 * (1 -net.gen[i].ramp_down ) * -net.gen[i].power_level[t][k])
                linear_terms[return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1)] = linear_terms[return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1)] + (2 * (1 -net.gen[i].ramp_down ) * net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)])
                quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,k,t))] + (net.gen[i].power_level[t][k]* net.gen[i].power_level[t][k])
                quadratic_terms[(return_var_name(i,l,t-1),return_var_name(i,l,t-1))] = quadratic_terms[(return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1),return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1))] + (net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)]* net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)])
                quadratic_terms[(return_var_name(i,k,t),return_var_name(i,l,t-1))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(i,min(l,len(net.gen[i].power_level[t-1])-1),t-1))] - 2*(net.gen[i].power_level[t][min(l,len(net.gen[i].power_level[t-1])-1)]* net.gen[i].power_level[t][k])
for t in range(1, net.time_period):
    constant = constant + (int(net.demand[t] + net.reserves[t] + 1) * int(net.demand[t] + net.reserves[t] + 1))
    for i in range(0, len(net.gen)):
        for k in range(0, len(net.gen[i].power_level[t])):    
            linear_terms[return_var_name(i,k,t)] = linear_terms[return_var_name(i,k,t)] - (2 * int(net.demand[t] + net.reserves[t] + 1) * -net.gen[i].power_level[t][k])
            for j in range(0, i):
                for z in range(0, min(len(net.gen[j].power_level[t]),k)):
                    quadratic_terms[(return_var_name(i,k,t),return_var_name(j,z,t))] = quadratic_terms[(return_var_name(i,k,t),return_var_name(j,z,t))] + (2*net.gen[i].power_level[t][k]*net.gen[j].power_level[t][z])

In [8]:
import dimod
from braket.aws import AwsDevice, AwsSession
from braket.ocean_plugin import BraketDWaveSampler
from dwave.system.composites import EmbeddingComposite
bqm = dimod.BinaryQuadraticModel(linear_terms, quadratic_terms, 'BINARY')
new_bqm = dimod.BinaryQuadraticModel(bqm.linear, {interaction: bias for interaction, bias in bqm.quadratic.items() if bias}, bqm.offset, bqm.vartype)

In [10]:
sampler_fin = BraketDWaveSampler(s3_destination_folder=(AwsSession().default_bucket(), 'd-wave/altawemd'),device_arn='arn:aws:braket:us-west-2::device/qpu/d-wave/Advantage_system6')
sampler = EmbeddingComposite(sampler_fin)
sampleset = sampler.sample(new_bqm)